In [ ]:
import pandas as pd
import os
import sqlite3

os.chdir('../')

In [ ]:
conn = sqlite3.connect('data/clients.db')
df = pd.read_sql_query("SELECT * FROM clients_processed", conn)

In [3]:
missed_matches = pd.read_csv('data/missed_matches.csv')

In [4]:
missed_matches

,id_a,id_b
0,80b1a124-693c-471e-bc90-8921c0eed8b1,c9be8d68-ba89-4c43-a651-a88ea6630045
1,440c61a4-a68a-47e6-a9a9-4324649eb4bd,fe2a2428-611b-4d7b-a818-f02c73d6cc92
2,59650c1d-2808-4b37-92f0-b66a85f1b26b,80b1a124-693c-471e-bc90-8921c0eed8b1
3,84ceb42a-714e-4126-b411-809d63ef2156,9efb6273-dcfc-40fb-b5cf-e01e558439eb
4,6cffaeda-04c0-4d1d-ab82-a45545836325,9869d1cc-11c8-4ad7-a343-062f5a7f1365
5,143d709f-2279-4f90-bf98-dd78ae71511d,9869d1cc-11c8-4ad7-a343-062f5a7f1365
6,9869d1cc-11c8-4ad7-a343-062f5a7f1365,ee807e49-126d-4f35-b246-422ab8a5e1be
7,9efb6273-dcfc-40fb-b5cf-e01e558439eb,baf6d6cc-bf0a-46b2-8d1e-b3d99ed18e5d


In [ ]:
# Join missed matches with client details for inspection
missed_details = missed_matches.merge(
    df[['record_id', 'first_name', 'last_name', 'dob', 'national_id', 'email']], 
    left_on='id_a', 
    right_on='record_id', 
    suffixes=('', '_a')
).merge(
    df[['record_id', 'first_name', 'last_name', 'dob', 'national_id', 'email']], 
    left_on='id_b', 
    right_on='record_id', 
    suffixes=('_a', '_b')
)

# Drop redundant ID columns and display
missed_details = missed_details.drop(columns=['record_id_a', 'record_id_b'])
missed_details


,id_a,id_b,first_name_a,last_name_a,dob_a,national_id_a,email_a,first_name_b,last_name_b,dob_b,national_id_b,email_b
0,80b1a124-693c-471e-bc90-8921c0eed8b1,c9be8d68-ba89-4c43-a651-a88ea6630045,LisN,Barxes,30/09/1968,548-50-3086,None,Lisa,Barnes,1968-09-30,272-51-2145,sarah35@example.org
1,440c61a4-a68a-47e6-a9a9-4324649eb4bd,fe2a2428-611b-4d7b-a818-f02c73d6cc92,Hvelmtrud,Kadx,1997-07-27,1206119O1W569,helmtrud.kade@hotmaiN.com,Helmtrud,Kade,1997-07-27,12061191W569,helmtrud.kade@hotmail.com
2,59650c1d-2808-4b37-92f0-b66a85f1b26b,80b1a124-693c-471e-bc90-8921c0eed8b1,Lisa,BarInes,1968-09-30,272-51-2145,sarah35@example.org,LisN,Barxes,30/09/1968,548-50-3086,None
3,84ceb42a-714e-4126-b411-809d63ef2156,9efb6273-dcfc-40fb-b5cf-e01e558439eb,E.,Butte,1960-05-24,74180102M183,None,EAeline,Btute,1960-05-24,None,None
4,6cffaeda-04c0-4d1d-ab82-a45545836325,9869d1cc-11c8-4ad7-a343-062f5a7f1365,Marek,Kilar,06-24-1960,20120558650,natan96@example.net,Marek,iKlar,1960-06-24,None,None
5,143d709f-2279-4f90-bf98-dd78ae71511d,9869d1cc-11c8-4ad7-a343-062f5a7f1365,Marek,Kilar,1960-06-24,20120558650,natan96@example.net,Marek,iKlar,1960-06-24,None,None
6,9869d1cc-11c8-4ad7-a343-062f5a7f1365,ee807e49-126d-4f35-b246-422ab8a5e1be,Marek,iKlar,1960-06-24,None,None,Marek,Kilar,1960-06-24,20120558650,None
7,9efb6273-dcfc-40fb-b5cf-e01e558439eb,baf6d6cc-bf0a-46b2-8d1e-b3d99ed18e5d,EAeline,Btute,1960-05-24,None,None,Eveline,Butte,1960-05-24,74180102M183,tdrub@example.com


In [6]:
missed_details.to_json()

'{"id_a":{"0":"80b1a124-693c-471e-bc90-8921c0eed8b1","1":"440c61a4-a68a-47e6-a9a9-4324649eb4bd","2":"59650c1d-2808-4b37-92f0-b66a85f1b26b","3":"84ceb42a-714e-4126-b411-809d63ef2156","4":"6cffaeda-04c0-4d1d-ab82-a45545836325","5":"143d709f-2279-4f90-bf98-dd78ae71511d","6":"9869d1cc-11c8-4ad7-a343-062f5a7f1365","7":"9efb6273-dcfc-40fb-b5cf-e01e558439eb"},"id_b":{"0":"c9be8d68-ba89-4c43-a651-a88ea6630045","1":"fe2a2428-611b-4d7b-a818-f02c73d6cc92","2":"80b1a124-693c-471e-bc90-8921c0eed8b1","3":"9efb6273-dcfc-40fb-b5cf-e01e558439eb","4":"9869d1cc-11c8-4ad7-a343-062f5a7f1365","5":"9869d1cc-11c8-4ad7-a343-062f5a7f1365","6":"ee807e49-126d-4f35-b246-422ab8a5e1be","7":"baf6d6cc-bf0a-46b2-8d1e-b3d99ed18e5d"},"first_name_a":{"0":"LisN","1":"Hvelmtrud","2":"Lisa","3":"E.","4":"Marek","5":"Marek","6":"Marek","7":"EAeline"},"last_name_a":{"0":"Barxes","1":"Kadx","2":"BarInes","3":"Butte","4":"Kilar","5":"Kilar","6":"iKlar","7":"Btute"},"dob_a":{"0":"30\\/09\\/1968","1":"1997-07-27","2":"1968-09-30",

In [17]:
conn.close()